# Tree Methods Consulting Project - SOLUTION

You've been hired by a dog food company to try to predict why some batches of their dog food are spoiling much quicker than intended! Unfortunately this Dog Food company hasn't upgraded to the latest machinery, meaning that the amounts of the five preservative chemicals they are using can vary a lot, but which is the chemical that has the strongest effect? The dog food company first mixes up a batch of preservative that contains 4 different preservative chemicals (A,B,C,D) and then is completed with a "filler" chemical. The food scientists beelive one of the A,B,C, or D preservatives is causing the problem, but need your help to figure out which one!
Use Machine Learning with RF to find out which parameter had the most predicitive power, thus finding out which chemical causes the early spoiling! So create a model and then find out how you can decide which chemical is the problem!

* Pres_A : Percentage of preservative A in the mix
* Pres_B : Percentage of preservative B in the mix
* Pres_C : Percentage of preservative C in the mix
* Pres_D : Percentage of preservative D in the mix
* Spoiled: Label indicating whether or not the dog food batch was spoiled.
___

**Think carefully about what this problem is really asking you to solve. While we will use Machine Learning to solve this, it won't be with your typical train/test split workflow. If this confuses you, skip ahead to the solution code along walk-through!**
____

In [2]:
import findspark 
findspark.init('/home/anshul/spark-3.0.0-preview2-bin-hadoop2.7/')

### create spark session

In [3]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('tree_method').getOrCreate()

### read the data and read the columns

In [4]:
data=spark.read.csv('dog_food.csv',inferSchema=True,header=True)

In [6]:
data.printSchema()

root
 |-- A: integer (nullable = true)
 |-- B: integer (nullable = true)
 |-- C: double (nullable = true)
 |-- D: integer (nullable = true)
 |-- Spoiled: double (nullable = true)



In [7]:
data.columns

['A', 'B', 'C', 'D', 'Spoiled']

In [8]:
data.show()

+---+---+----+---+-------+
|  A|  B|   C|  D|Spoiled|
+---+---+----+---+-------+
|  4|  2|12.0|  3|    1.0|
|  5|  6|12.0|  7|    1.0|
|  6|  2|13.0|  6|    1.0|
|  4|  2|12.0|  1|    1.0|
|  4|  2|12.0|  3|    1.0|
| 10|  3|13.0|  9|    1.0|
|  8|  5|14.0|  5|    1.0|
|  5|  8|12.0|  8|    1.0|
|  6|  5|12.0|  9|    1.0|
|  3|  3|12.0|  1|    1.0|
|  9|  8|11.0|  3|    1.0|
|  1| 10|12.0|  3|    1.0|
|  1|  5|13.0| 10|    1.0|
|  2| 10|12.0|  6|    1.0|
|  1| 10|11.0|  4|    1.0|
|  5|  3|12.0|  2|    1.0|
|  4|  9|11.0|  8|    1.0|
|  5|  1|11.0|  1|    1.0|
|  4|  9|12.0| 10|    1.0|
|  5|  8|10.0|  9|    1.0|
+---+---+----+---+-------+
only showing top 20 rows



### assemble the data

In [9]:
from pyspark.ml.linalg import Vector
from pyspark.ml.feature import VectorAssembler

In [10]:
assembler=VectorAssembler(inputCols=['A', 'B', 'C', 'D'],
                         outputCol='feature')

In [13]:
new_data=assembler.transform(data)

In [15]:
new_data.columns

['A', 'B', 'C', 'D', 'Spoiled', 'feature']

In [16]:
final_data=new_data.select(['feature','Spoiled'])

In [17]:
final_data.show()

+-------------------+-------+
|            feature|Spoiled|
+-------------------+-------+
| [4.0,2.0,12.0,3.0]|    1.0|
| [5.0,6.0,12.0,7.0]|    1.0|
| [6.0,2.0,13.0,6.0]|    1.0|
| [4.0,2.0,12.0,1.0]|    1.0|
| [4.0,2.0,12.0,3.0]|    1.0|
|[10.0,3.0,13.0,9.0]|    1.0|
| [8.0,5.0,14.0,5.0]|    1.0|
| [5.0,8.0,12.0,8.0]|    1.0|
| [6.0,5.0,12.0,9.0]|    1.0|
| [3.0,3.0,12.0,1.0]|    1.0|
| [9.0,8.0,11.0,3.0]|    1.0|
|[1.0,10.0,12.0,3.0]|    1.0|
|[1.0,5.0,13.0,10.0]|    1.0|
|[2.0,10.0,12.0,6.0]|    1.0|
|[1.0,10.0,11.0,4.0]|    1.0|
| [5.0,3.0,12.0,2.0]|    1.0|
| [4.0,9.0,11.0,8.0]|    1.0|
| [5.0,1.0,11.0,1.0]|    1.0|
|[4.0,9.0,12.0,10.0]|    1.0|
| [5.0,8.0,10.0,9.0]|    1.0|
+-------------------+-------+
only showing top 20 rows



### using the random forest

In [34]:
from pyspark.ml.classification import (
RandomForestClassifier,DecisionTreeClassifier)

In [38]:
rfc=RandomForestClassifier(labelCol='Spoiled',featuresCol='feature')

In [48]:
rfc_model=rfc.fit(final_data)
final_data.head(1)

[Row(feature=DenseVector([4.0, 2.0, 12.0, 3.0]), Spoiled=1.0)]

In [52]:
rfc_model.featureImportances

SparseVector(4, {0: 0.0232, 1: 0.0156, 2: 0.9418, 3: 0.0194})

### it shows that the chemical at index 2 (chemical C) is the most important feature, therefore it is causing the early damage.

#### Decision Tree Classifier as well

In [56]:
dtc=DecisionTreeClassifier(labelCol='Spoiled',featuresCol='feature',
                          maxDepth=10)

In [57]:
dtc_model=dtc.fit(final_data)
final_data.head(1)

[Row(feature=DenseVector([4.0, 2.0, 12.0, 3.0]), Spoiled=1.0)]

In [58]:
dtc_model.featureImportances

SparseVector(4, {0: 0.0152, 1: 0.0137, 2: 0.9327, 3: 0.0384})

### it shows that the chemical at index 2 (chemical C) is the most important feature, therefore it is causing the early damage.